In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt
import pickle

maxsize = 10000
out_dir = "../dataset"
save_file = out_dir + "/dataset.pickle" # 保存先


result = []
fs = glob.glob(out_dir+"/*")
for i,labeldir in enumerate(fs):
    print("i=",i)
    print("labeldir=",labeldir)
    fs0 = glob.glob(labeldir+"/*")
    for j,f in enumerate(fs0):
        if j>=maxsize:break
        result.append([i,cv2.imread(f)])

pickle.dump(result, open(save_file, "wb"))
print("done!")

In [ ]:
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

num_classes = 7
im_rows = 80
im_cols = 60
im_color = 3
in_shape = (im_rows, im_cols, im_color)

In [ ]:
data_file =  "../dataset/dataset.pickle"
data = pickle.load(open(data_file,"rb"))

In [ ]:
X = []
y = []
for d in data:
    (label,img)=d

    y.append(label)
    X.append(img)

X = np.array(X)
y = np.array(y)

In [ ]:
# train dataとtest dataに分類
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8, test_size=0.2,shuffle=True)

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

#One-Hot形式前のy_testを保存
y_test_org = y_test

# ラベルデータをOne-Hot形式に変換
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)


In [ ]:
from keras.layers import RandomRotation, RandomZoom

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                    input_shape=in_shape))

    #水増し処理を追加
    # model.add(RandomTranslation(0.1, 0.1))
    model.add(RandomRotation(0.2))
    model.add(RandomZoom(0.3))

    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    model.summary()

    model.compile(
        loss='categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy'])

    return model

In [ ]:
model = build_model()

In [ ]:
hist = model.fit(X_train, y_train,
    batch_size=128, epochs=30,
    verbose=1,
    validation_data=(X_test, y_test))

In [ ]:
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def model_evaluation(model):
    score = model.evaluate(X_test, y_test, verbose=1)
    print('正解率=', score[1], 'loss=', score[0])

    pred = model.predict(X_test) # modelは学習させたもの
    correct_count = [0] * num_classes
    count = [0] * num_classes

    for i in range(pred.shape[0]):
        prediction = np.argmax(pred[i]) # モデルの予測ラベル取得
        answer = y_test_org[i]
        count[answer] += 1
        if prediction == answer:
            correct_count[answer] += 1 # 正解数カウント

    accuracy = [correct/N for correct, N in zip(correct_count, count)] # 精度算出

    for label, acc in enumerate(accuracy):
        print('accuracy for label {} : {}'.format(label, acc))
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('Accuracy')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Loss')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
model_evaluation(model)

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, mean_absolute_error
import seaborn as sns

In [ ]:
kf = KFold(n_splits=3, shuffle=True)

all_loss=[]
all_val_loss=[]
all_acc=[]
all_val_acc=[]
ep=20

for train_index, val_index in kf.split(X_train,y_train):

    train_data=X_train[train_index]
    train_label=y_train[train_index]
    val_data=X_train[val_index]
    val_label=y_train[val_index]

    model=build_model()
    history=model.fit(train_data,
                      train_label,
                      epochs=ep,
                      batch_size=128,
                      verbose = 1,
                      validation_data=(val_data,val_label))

    loss=history.history['loss']
    val_loss=history.history['val_loss']
    acc=history.history['accuracy']
    val_acc=history.history['val_accuracy']

    all_loss.append(loss)
    all_val_loss.append(val_loss)
    all_acc.append(acc)
    all_val_acc.append(val_acc)

    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Accuracy')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()


ave_all_loss=[
    np.mean([x[i] for x in all_loss]) for i in range(ep)]
ave_all_val_loss=[
    np.mean([x[i] for x in all_val_loss]) for i in range(ep)]
ave_all_acc=[
    np.mean([x[i] for x in all_acc]) for i in range(ep)]
ave_all_val_acc=[
    np.mean([x[i] for x in all_val_acc]) for i in range(ep)]

In [ ]:
preds = []
pred = model.predict(X_test)
y_pred = np.mean(pred, axis=0)
print(y_pred)
print(f'Test Score MAE: {mean_absolute_error(pred, y_test)}')

In [ ]:
y_test = np.argmax(y_test, axis=1)

In [ ]:
pred = np.argmax(model.predict(X_test), axis=1)
plt.figure(figsize=(30,15))
cm = confusion_matrix(y_test, pred)
sns.heatmap(cm,  annot=True, fmt='d')
plt.show()